<a href="https://colab.research.google.com/github/ArpanSM/Object-Detection-and-Instance-Segmentaion/blob/master/YOLO_v3_Multiple_Class_Object_Detection_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone DarkNet Repo

In [0]:
!git clone https://github.com/pjreddie/darknet

In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

## Download Yolo weights and Darknet Model

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

## Get Data from Drive

Format of Data


*   Make a folder - `custom_data`
*   Folder should contain : 
    *   `detector.data` - contains - classes, train, test, names and backup
    *   `text.txt` 
    *   `train.txt`
    *   `custom.names` - name of all classes.
    *   `images` - folder to store all images and labels
    *   `cfg` - contains the config file




In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!cp -avr /content/drive/My\ Drive/yolo_v31/custom_data /content/

In [0]:
cd /content/custom_data/images 

In [0]:
!unzip /content/drive/My\ Drive/yolo_v31/images.zip

In [0]:
#make a label folder to store labels files separately
!mkdir /content/custom_data/labels
import os, shutil, glob
src_fldr = r"/content/custom_data/images/"; ## Edit this
dst_fldr = "/content/custom_data/labels/"; ## Edit this
for txt_file in glob.glob(src_fldr+"*.txt"):
    shutil.copy2(txt_file, dst_fldr);

#### Configuration files setting alternate method

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp /content/darknet/cfg/yolov3.cfg /content/custom_data/cfg/yolov3_training.cfg

In [0]:
#edit the batch_size
!sed -i 's/batch=1/batch=64/' /content/custom_data/cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' /content/custom_data/cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' /content/custom_data/cfg/yolov3_training.cfg
#change the number of classes as required from 80 --> x
!sed -i '610 s@classes=80@classes=1@' /content/custom_data/cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' /content/custom_data/cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' /content/custom_data/cfg/yolov3_training.cfg
#change the number of filters as (classes*3)+1
!sed -i '603 s@filters=255@filters=18@' /content/custom_data/cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' /content/custom_data/cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' /content/custom_data/cfg/yolov3_training.cfg

## Train

In [0]:
cd /content/darknet/

/content/darknet


In [0]:
!./darknet detector train /content/custom_data/detector.data /content/custom_data/cfg/yolov3-custom.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Region 106 Avg IOU: 0.434686, Class: 0.635082, Obj: 0.662354, No Obj: 0.000420, .5R: 0.333333, .75R: 0.000000,  count: 3
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000008, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000016, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: 0.613037, Class: 0.913363, Obj: 0.429606, No Obj: 0.000786, .5R: 0.833333, .75R: 0.000000,  count: 6
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000009, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000006, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: 0.627668, Class: 0.910077, Obj: 0.640707, No Obj: 0.000559, .5R: 0.666667, .75R: 0.333333,  count: 3
Region 82 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000008, .5R: -nan, .75R: -nan,  count: 0
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000007, .5R: -nan, .75R: -n

In [0]:
!cp -avr /content/yolov3-custom_900.weights /content/drive/My\ Drive/yolo_v31/

'/content/yolov3-custom_900.weights' -> '/content/drive/My Drive/yolo_v31/yolov3-custom_900.weights'


##Test

In [0]:

import argparse

import cv2
import numpy as np

parser = argparse.ArgumentParser(add_help=False)
parser.add_argument("--image", default='samples/image.jpg', help="image for prediction")
parser.add_argument("--config", default='cfg/yolov3.cfg', help="YOLO config path")
parser.add_argument("--weights", default='yolov3.weights', help="YOLO weights path")
parser.add_argument("--names", default='data/coco.names', help="class names path")
args = parser.parse_args()

CONF_THRESH, NMS_THRESH = 0.5, 0.5

# Load the network
net = cv2.dnn.readNetFromDarknet(args.config, args.weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Get the output layer from YOLO
layers = net.getLayerNames()
output_layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Read and convert the image to blob and perform forward pass to get the bounding boxes with their confidence scores
img = cv2.imread(args.image)
height, width = img.shape[:2]

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_outputs = net.forward(output_layers)

class_ids, confidences, b_boxes = [], [], []
for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > CONF_THRESH:
            center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            b_boxes.append([x, y, int(w), int(h)])
            confidences.append(float(confidence))
            class_ids.append(int(class_id))

# Perform non maximum suppression for the bounding boxes to filter overlapping and low confident bounding boxes
indices = cv2.dnn.NMSBoxes(b_boxes, confidences, CONF_THRESH, NMS_THRESH).flatten().tolist()

# Draw the filtered bounding boxes with their class to the image
with open(args.names, "r") as f:
    classes = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for index in indices:
    x, y, w, h = b_boxes[index]
    cv2.rectangle(img, (x, y), (x + w, y + h), colors[index], 2)
    cv2.putText(img, classes[class_ids[index]], (x + 5, y + 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, colors[index], 2)

cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Kaggle

In [0]:
import numpy as np 
import pandas as pd 
import cv2
import os
import matplotlib.pyplot as plt

#Reading Image 00002.jpg and getting its spatial dimension
image_BGR = cv2.imread('/content/custom_data/images/00002.jpg')
print('Image shape:', image_BGR.shape)
h, w = image_BGR.shape[:2]  
print('Image height={0} and width={1}'.format(h, w))

In [0]:
#Drawing Bounding Boxes of Traffic Signs on Image

with open('/content/custom_data/images/00002.txt') as f:
    # Preparing list for annotation of BB (bounding boxes)
    lst = []
    for line in f:
        lst += [line.rstrip()]
        print(line)

# Going through all BB
for i in range(len(lst)):
    # Getting current bounding box coordinates, its width and height
    bb_current = lst[i].split()
    x_center, y_center = int(float(bb_current[1]) * w), int(float(bb_current[2]) * h)
    box_width, box_height = int(float(bb_current[3]) * w), int(float(bb_current[4]) * h)
    
    # Now, from YOLO data format, we can get top left corner coordinates
    # that are x_min and y_min
    x_min = int(x_center - (box_width / 2))
    y_min = int(y_center - (box_height / 2))

    # Drawing bounding box on the original image
    cv2.rectangle(image_BGR, (x_min, y_min), (x_min + box_width, y_min + box_height), [172 , 10, 127], 2)

    # Preparing text with label and confidence for current bounding box
    class_current = 'Class: {}'.format(bb_current[0])

    # Putting text with label and confidence on the original image
    cv2.putText(image_BGR, class_current, (x_min, y_min - 5), cv2.FONT_HERSHEY_COMPLEX, 0.7, [172 , 10, 127], 2)

In [0]:
#Showing Image 00002.jpg with Detected Traffic Signs
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 15)
fig = plt.figure()
plt.imshow(cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Image 00003.jpg with Traffic Signs', fontsize=18)
plt.show()
fig.savefig('example.png')
plt.close()